In [51]:
from lbl2vec import Lbl2Vec
import pandas as pd
from gensim.utils import simple_preprocess
from gensim.models.doc2vec import TaggedDocument
from gensim.parsing.preprocessing import strip_tags
from sklearn.metrics import f1_score

### Data Load

In [52]:
train_org = pd.read_excel("C:\\Users\\yukir\\Documents\Monicas_workspace\\Derma\\important_data\\★★[0518]derma_train_v2.xlsx",index_col=0)
test_org = pd.read_excel("C:\\Users\\yukir\\Documents\Monicas_workspace\\Derma\\important_data\\★★[0518]derma_test_v2.xlsx",index_col=0)

labels = pd.read_csv("C:\\Users\\yukir\\Documents\\Monicas_workspace\\Derma\\important_data\\(0519)labels.csv",sep=';', encoding='cp949')

# labels = pd.read_csv("C:\\Users\\yukir\\Documents\\Monicas_workspace\\Derma\\important_data\\★★[0518]servqual_labels.xlsx", sep =';')
# split keywords by separator and save them as array
labels['keywords'] = labels['keywords'].apply(lambda x: x.split(' '))

# convert description keywords to lowercase
labels['keywords'] = labels['keywords'].apply(lambda description_keywords: [keyword.lower() for keyword in description_keywords])

# get number of keywords for each class
labels['number_of_keywords'] = labels['keywords'].apply(lambda row: len(row))

In [53]:
ag_train = train_org.copy()
ag_test = test_org.copy()

In [54]:
labels

,class_index,class_name,keywords,number_of_keywords
0,1,신뢰성,"[치료, 시술, 처방, 주사, 효과, 가격, 제거, 관리, 레이저, 피부관리, 과잉...",28
1,2,반응성,"[대기, 예약, 이벤트, 응대, 전화, 먼저, 일찍, 답변, 질문, 당일, 검색, ...",21
2,3,확신성,"[선생님, 의사, 원장님, 직원, 간호사, 데스크, 실장, 원장, 안내, 접수, 전...",23
3,4,공감성,"[설명, 상담, 친절, 말씀, 상세, 도움, 느낌, 주의, 권유, 이해, 이야기, ...",23
4,5,유형성',"[시설, 내부, 근처, 건물, 위치, 분위기, 주차, 주변, 대기실, 주차장, 접근...",29


In [55]:
ag_train.head()

,hospital,reviews,class_index
223312,닥터쁘띠의원 강남1호점,접수하고 거의 바로 상담받았어요,4
136163,서울미의원 본점,생활 습관이나 관리 방법에 대해 서도 일절 언급 은 없으셨습니다,1
178066,WS라인의원,일주일 후에 결과 들으러,1
189394,행복한피부과의원,예약 안 하고,2
120973,고운세상피부과의원,여드름 약물 치료 딸 여드름 치료 상담받았습니다,4


### Tokenizeing

In [56]:
# doc: document text string
# returns tokenized document
# strip_tags removes meta tags from the text
# simple preprocess converts a document into a list of lowercase tokens, ignoring tokens that are too short or too long 
# simple preprocess also removes numerical values as well as punktuation characters
def tokenize(doc):
    return simple_preprocess(strip_tags(doc), deacc=True, min_len=2, max_len=15)

In [57]:
# add data set type column
ag_train['data_set_type'] = 'train'
ag_test['data_set_type'] = 'test'

# concat train and test data
ag_full_corpus = pd.concat([ag_train,ag_test]).reset_index(drop=True)

In [58]:
ag_full_corpus.head()

,hospital,reviews,class_index,data_set_type
0,닥터쁘띠의원 강남1호점,접수하고 거의 바로 상담받았어요,4,train
1,서울미의원 본점,생활 습관이나 관리 방법에 대해 서도 일절 언급 은 없으셨습니다,1,train
2,WS라인의원,일주일 후에 결과 들으러,1,train
3,행복한피부과의원,예약 안 하고,2,train
4,고운세상피부과의원,여드름 약물 치료 딸 여드름 치료 상담받았습니다,4,train


In [59]:
# tokenize and tag documents combined title + description for Lbl2Vec training
ag_full_corpus['tagged_docs'] = ag_full_corpus.apply(lambda row: TaggedDocument(tokenize(row['reviews']), [str(row.name)]), axis=1)

In [60]:
ag_full_corpus.head()

,hospital,reviews,class_index,data_set_type,tagged_docs
0,닥터쁘띠의원 강남1호점,접수하고 거의 바로 상담받았어요,4,train,"([접수하고, 거의, 바로, 상담받았어요], [0])"
1,서울미의원 본점,생활 습관이나 관리 방법에 대해 서도 일절 언급 은 없으셨습니다,1,train,"([생활, 습관이나, 관리, 방법에, 대해, 서도, 일절, 언급, 없으셨습니다], ..."
2,WS라인의원,일주일 후에 결과 들으러,1,train,"([일주일, 후에, 결과, 들으러], [2])"
3,행복한피부과의원,예약 안 하고,2,train,"([예약, 하고], [3])"
4,고운세상피부과의원,여드름 약물 치료 딸 여드름 치료 상담받았습니다,4,train,"([여드름, 약물, 치료, 여드름, 치료, 상담받았습니다], [4])"


In [61]:
ag_full_corpus.tagged_docs

0                             ([접수하고, 거의, 바로, 상담받았어요], [0])
1         ([생활, 습관이나, 관리, 방법에, 대해, 서도, 일절, 언급, 없으셨습니다], ...
2                                 ([일주일, 후에, 결과, 들으러], [2])
3                                           ([예약, 하고], [3])
4                    ([여드름, 약물, 치료, 여드름, 치료, 상담받았습니다], [4])
                                ...                        
171747                   ([이렇게, 부연, 설명을, 덧붙여주시긴], [171747])
171748                            ([여기만큼, 친절하시고], [171748])
171749        ([또한, 진료, 이동, 후에도, 의사, 선생님께서, 오셔서], [171749])
171750    ([인중, 겨드랑이, 겨드랑이, 인중, 종아리, 레이저, 제모, 권종아리, 만해도,...
171751                    ([제거, 병원, 접수부터, 안내하시는], [171751])
Name: tagged_docs, Length: 171752, dtype: object

In [62]:
# add doc_key column
ag_full_corpus['doc_key'] = ag_full_corpus.index.astype(str)

In [73]:
labels

,class_index,class_name,keywords,number_of_keywords
0,1,신뢰성,"[치료, 시술, 처방, 주사, 효과, 가격, 제거, 관리, 레이저, 피부관리, 과잉...",28
1,2,반응성,"[대기, 예약, 이벤트, 응대, 전화, 먼저, 일찍, 답변, 질문, 당일, 검색, ...",21
2,3,확신성,"[선생님, 의사, 원장님, 직원, 간호사, 데스크, 실장, 원장, 안내, 접수, 전...",23
3,4,공감성,"[설명, 상담, 친절, 말씀, 상세, 도움, 느낌, 주의, 권유, 이해, 이야기, ...",23
4,5,유형성',"[시설, 내부, 근처, 건물, 위치, 분위기, 주차, 주변, 대기실, 주차장, 접근...",29


In [69]:
ag_full_corpus

,hospital,reviews,class_index,data_set_type,tagged_docs,doc_key
0,닥터쁘띠의원 강남1호점,접수하고 거의 바로 상담받았어요,4,train,"([접수하고, 거의, 바로, 상담받았어요], [0])",0
1,서울미의원 본점,생활 습관이나 관리 방법에 대해 서도 일절 언급 은 없으셨습니다,1,train,"([생활, 습관이나, 관리, 방법에, 대해, 서도, 일절, 언급, 없으셨습니다], ...",1
2,WS라인의원,일주일 후에 결과 들으러,1,train,"([일주일, 후에, 결과, 들으러], [2])",2
3,행복한피부과의원,예약 안 하고,2,train,"([예약, 하고], [3])",3
4,고운세상피부과의원,여드름 약물 치료 딸 여드름 치료 상담받았습니다,4,train,"([여드름, 약물, 치료, 여드름, 치료, 상담받았습니다], [4])",4
...,...,...,...,...,...,...
171747,닥터스피부과의원 마포공덕점,뭐 이렇게 부연 설명을 덧붙여주시긴,4,test,"([이렇게, 부연, 설명을, 덧붙여주시긴], [171747])",171747
171748,타임피부과의원,여기만큼 친절하시고,4,test,"([여기만큼, 친절하시고], [171748])",171748
171749,하얀나라피부과의원,또한 진료 실 이동 후에도 의사 선생님께서 오셔서,3,test,"([또한, 진료, 이동, 후에도, 의사, 선생님께서, 오셔서], [171749])",171749
171750,샤인빔의원 노원점,인중 제 모 겨드랑이 제 모 겨드랑이 인중 종아리 레이저 제모 5회 권종아리 1회 ...,4,test,"([인중, 겨드랑이, 겨드랑이, 인중, 종아리, 레이저, 제모, 권종아리, 만해도,...",171750


In [70]:
ag_full_corpus['class_index'] = ag_full_corpus['class_index'].astype(str)

In [74]:
labels['class_index'] = labels['class_index'].astype(str)

In [75]:
# add class_name column
ag_full_corpus = ag_full_corpus.merge(labels, left_on='class_index', right_on='class_index', how='left').drop(['class_index', 'keywords'], axis=1)

In [76]:
ag_full_corpus.head()

,hospital,reviews,data_set_type,tagged_docs,doc_key,class_name,number_of_keywords
0,닥터쁘띠의원 강남1호점,접수하고 거의 바로 상담받았어요,train,"([접수하고, 거의, 바로, 상담받았어요], [0])",0,공감성,23
1,서울미의원 본점,생활 습관이나 관리 방법에 대해 서도 일절 언급 은 없으셨습니다,train,"([생활, 습관이나, 관리, 방법에, 대해, 서도, 일절, 언급, 없으셨습니다], ...",1,신뢰성,28
2,WS라인의원,일주일 후에 결과 들으러,train,"([일주일, 후에, 결과, 들으러], [2])",2,신뢰성,28
3,행복한피부과의원,예약 안 하고,train,"([예약, 하고], [3])",3,반응성,21
4,고운세상피부과의원,여드름 약물 치료 딸 여드름 치료 상담받았습니다,train,"([여드름, 약물, 치료, 여드름, 치료, 상담받았습니다], [4])",4,공감성,23


In [77]:
labels.keywords

0    [치료, 시술, 처방, 주사, 효과, 가격, 제거, 관리, 레이저, 피부관리, 과잉...
1    [대기, 예약, 이벤트, 응대, 전화, 먼저, 일찍, 답변, 질문, 당일, 검색, ...
2    [선생님, 의사, 원장님, 직원, 간호사, 데스크, 실장, 원장, 안내, 접수, 전...
3    [설명, 상담, 친절, 말씀, 상세, 도움, 느낌, 주의, 권유, 이해, 이야기, ...
4    [시설, 내부, 근처, 건물, 위치, 분위기, 주차, 주변, 대기실, 주차장, 접근...
Name: keywords, dtype: object

### Train Lbl2Vec

In [ ]:
# init model with parameters
# 실제로 데이터셋에서 labeled class를 쓰진않음
lbl2vec_model = Lbl2Vec(keywords_list=list(labels['keywords']), tagged_documents=ag_full_corpus['tagged_docs'][ag_full_corpus['data_set_type']=='train'], label_names=list(labels['class_name']), similarity_threshold=0.10, min_num_docs=1, epochs=1000)

In [138]:
# train model
lbl2vec_model.fit()

2022-05-19 22:16:58,855 - Lbl2Vec - INFO - Train document and word embeddings


KeyboardInterrupt: 

### Predict

In [ ]:
# predict similarity scores
model_docs_lbl_similarities = lbl2vec_model.predict_model_docs()

2022-05-19 22:11:07,827 - Lbl2Vec - INFO - Get document embeddings from model
2022-05-19 22:11:07,930 - Lbl2Vec - INFO - Calculate document<->label similarities


In [ ]:
model_docs_lbl_similarities.head()

,doc_key,most_similar_label,highest_similarity_score,신뢰성,반응성,확신성,공감성,유형성'
0,0,확신성,0.660759,0.660390,0.660365,0.660759,0.660356,0.660411
1,1,공감성,0.481859,0.481236,0.481248,0.481507,0.481859,0.480917
2,2,확신성,0.629692,0.629330,0.629312,0.629692,0.629615,0.629167
3,3,유형성',0.720253,0.720183,0.720187,0.720131,0.720024,0.720253
4,4,유형성',0.652010,0.651841,0.651842,0.651761,0.651275,0.652010


### Evaluation

In [ ]:
# merge DataFrames to compare the predicted and true topic labels
evaluation_train = model_docs_lbl_similarities.merge(ag_full_corpus[ag_full_corpus['data_set_type']=='train'], left_on='doc_key', right_on='doc_key')

In [ ]:
y_true_train = evaluation_train['class_name']
y_pred_train = evaluation_train['most_similar_label']
print('F1 score:',f1_score(y_true_train, y_pred_train, average='micro'))

F1 score: 0.2418869275758259


### Predict

# predict similarity scores of new test documents (they were not used during Lbl2Vec training)
new_docs_lbl_similarities = lbl2vec_model.predict_new_docs(tagged_docs=ag_full_corpus['tagged_docs'][ag_full_corpus['data_set_type']=='test'])

In [ ]:
# predict similarity scores of new test documents (they were not used during Lbl2Vec training)
new_docs_lbl_similarities = lbl2vec_model.predict_new_docs(tagged_docs=ag_full_corpus['tagged_docs'][ag_full_corpus['data_set_type']=='test'])

2022-05-19 22:14:05,237 - Lbl2Vec - INFO - Calculate document embeddings
2022-05-19 22:14:17,727 - Lbl2Vec - INFO - Calculate document<->label similarities


In [ ]:
new_docs_lbl_similarities.head()

,doc_key,most_similar_label,highest_similarity_score,신뢰성,반응성,확신성,공감성,유형성'
0,150806,유형성',0.480287,0.480024,0.480029,0.480266,0.479809,0.480287
1,150807,유형성',0.497677,0.497597,0.497568,0.497548,0.497501,0.497677
2,150808,유형성',0.454256,0.454084,0.454095,0.454004,0.453635,0.454256
3,150809,확신성,0.459040,0.458233,0.458248,0.459040,0.458341,0.458512
4,150810,공감성,0.492284,0.492221,0.492230,0.491712,0.492284,0.492130


In [ ]:
# merge DataFrames to compare the predicted and true topic labels
evaluation_test = new_docs_lbl_similarities.merge(ag_full_corpus[ag_full_corpus['data_set_type']=='test'], left_on='doc_key', right_on='doc_key')

In [ ]:
y_true_test = evaluation_test['class_name']
y_pred_test = evaluation_test['most_similar_label']
print('F1 score:',f1_score(y_true_test, y_pred_test, average='micro'))

F1 score: 0.2751838059772749


In [ ]:
y_truelist = y_true_test.tolist()
y_pred_list = y_pred_test.tolist()

In [ ]:
ag_test['y_true'] = y_truelist
ag_test['y_pred']= y_pred_list

ag_test

,hospital,reviews,class_index,data_set_type,y_pred,y_true
56388,서울림피부과의원,그 사람 도 만족 만족 하하하 원장님 이 친절하고,3,test,유형성',확신성
45149,상상의원 신사본점,시술 은 이건데,1,test,유형성',신뢰성
168707,예인피부과의원 광진점,염증 치료 대기는 거의 오픈 하 자마자 갔는데,1,test,유형성',신뢰성
206503,김영숙피부과의원,병원 리뷰 직원 이 친절하지는,3,test,확신성,확신성
211003,아비쥬의원 천호점,전에 의사 선생님 보고 얘기할,3,test,공감성,확신성
...,...,...,...,...,...,...
132874,닥터스피부과의원 마포공덕점,뭐 이렇게 부연 설명을 덧붙여주시긴,4,test,공감성,공감성
79475,타임피부과의원,여기만큼 친절하시고,4,test,확신성,공감성
185303,하얀나라피부과의원,또한 진료 실 이동 후에도 의사 선생님께서 오셔서,3,test,공감성,확신성
151977,샤인빔의원 노원점,인중 제 모 겨드랑이 제 모 겨드랑이 인중 종아리 레이저 제모 5회 권종아리 1회 ...,4,test,공감성,공감성
